In [2]:
import tensorflow as tf


#
# define earth mover distance (wasserstein loss)
#

LEARNING_RATE = 0.0002
BETA_1 = 0.5
BETA_2 = 0.999

In [3]:
batch_size = 32
def em_loss(y_coefficients, y_pred):
    return tf.reduce_mean(tf.multiply(y_coefficients, y_pred))

#
# construct computation graph for calculating the gradient penalty (improved wGAN) and training the discriminator
#

# sample a batch of noise (generator input)
_z = tf.placeholder(tf.float32, shape=(batch_size, rand_dim))

# sample a batch of real images
_x = tf.placeholder(tf.float32, shape=(batch_size, img_height, img_width, img_channels))

# generate a batch of images with the current generator
_g_z = generator_model(_z)

# calculate `x_hat`
epsilon = tf.placeholder(tf.float32, shape=(batch_size, 1, 1, 1))
x_hat = epsilon * _x + (1.0 - epsilon) * _g_z

# gradient penalty
gradients = tf.gradients(discriminator_model(x_hat), [x_hat])
_gradient_penalty = 10.0 * tf.square(tf.norm(gradients[0], ord=2) - 1.0)

# calculate discriminator's loss
_disc_loss = em_loss(tf.ones(batch_size), discriminator_model(_g_z)) - \
    em_loss(tf.ones(batch_size), discriminator_model(_x)) + \
    _gradient_penalty

NameError: name 'rand_dim' is not defined

In [6]:
def gradient_penalty(f, real, fake=None):
    def _interpolate(a, b):
        with tf.name_scope('interpolate'):
            shape = [tf.shape(a)[0]] + [1] * (a.shape.ndims - 1)
            alpha = tf.random_uniform(shape=shape, minval=0., maxval=1.)
            inter = a + alpha * (b - a)
            inter.set_shape(a.get_shape().as_list())
            return inter

    with tf.name_scope('gradient_penalty'):
        x = _interpolate(real, fake)
        pred = f(x) #discriminator
        if isinstance(pred, tuple):
            pred = pred[0]
        grad = tf.gradients(pred, x)[0]
        norm = tf.norm(slim.flatten(grad), axis=1)
        gp = tf.reduce_mean((norm - 1.)**2)
        return gp

<tf.Tensor 'random_uniform_1:0' shape=(32, 1, 1, 1) dtype=float32>

In [ ]:
 if self.gan_type == 'wgan-gp':
            epsilon = tf.random_uniform(
                shape=[self.batch_size, 1, 1, 1], minval=0., maxval=1.)
            interpolated_image = self.image + epsilon * (fake_image - self.image)
            d_interpolated = D(interpolated_image)
        # }}}

        # Build losses {{{
        # =========
        # compute loss and prob
        if self.gan_type == 'lsgan':
            d_real_loss = tf.reduce_mean((d_real - tf.ones_like(d_real))**2)
            d_fake_loss = tf.reduce_mean((d_fake - tf.zeros_like(d_fake))**2)
            g_loss = tf.reduce_mean((d_fake - tf.ones_like(d_fake))**2)
        elif self.gan_type == 'hinge':
            d_real_loss = tf.reduce_mean(tf.nn.relu(tf.ones_like(d_real) - d_real))
            d_fake_loss = tf.reduce_mean(tf.nn.relu(tf.ones_like(d_fake) + d_fake))
            g_loss = -tf.reduce_mean(d_fake)
        elif self.gan_type == 'wgan-gp':
            d_loss = tf.reduce_mean(d_fake) - tf.reduce_mean(d_real)
            g_loss = -tf.reduce_mean(d_fake)
        else:
            raise NotImplementedError

        d_real_prob = tf.reduce_mean(d_real)
        d_fake_prob = tf.reduce_mean(d_fake)

        # compute gradient penalty
        if self.gan_type == 'wgan-gp':
            grad_d_interpolated = tf.gradients(
                d_interpolated, [interpolated_image])[0]
            slopes = tf.sqrt(1e-8 + tf.reduce_sum(
                tf.square(grad_d_interpolated), axis=[1, 2, 3]))
            gradient_penalty = tf.reduce_mean((slopes - 1.) ** 2)

        if self.gan_type in ['lsgan', 'hinge']:
            d_loss = d_real_loss + d_fake_loss
        self.d_loss = d_loss
        self.g_loss = g_loss
        if self.gan_type == 'wgan-gp':
            self.d_loss += self.gamma * gradient_penalty
            tf.summary.scalar("loss/gradient_penalty", gradient_penalty)
        # }}}

        # TensorBoard summaries {{{
        # =========
        if self.gan_type == 'lsgan':
            tf.summary.scalar("loss/d_real_loss", d_real_loss)
            tf.summary.scalar("loss/d_fake_loss", d_fake_loss)
        tf.summary.scalar("loss/d_real_prob", d_real_prob)
        tf.summary.scalar("loss/d_fake_prob", d_fake_prob)
        tf.summary.scalar("loss/d_loss", self.d_loss)
        tf.summary.scalar("loss/g_loss", self.g_loss)

        def d_output_vis(d_output):
            d_vis = tf.tile(tf.image.resize_nearest_neighbor(
                tf.clip_by_value(d_output, -1, 1),
                [self.h, self.w]), [1, 1, 1, self.c_dim])
            return d_vis

        tb_d = tf.concat([d_output_vis(d_real), d_output_vis(d_fake)], axis=2)
        if self.gan_type == 'wgan-gp':
            # normlize to [-1, 1]
            tb_d -= tf.reduce_min(tb_d)
            tb_d /= tf.reduce_max(tb_d)
            tb_d = tb_d * 2 - 1
        tb_img = tf.concat([self.image, fake_image], axis=2)
        tb_image = tf.concat([tb_img, tb_d], axis=1)
        tf.summary.image("img", tb_image)